In [ ]:
from ompl_demo import create_state_checker_env, INTERPOLATE_NUM, DEFAULT_PLANNING_TIME
from utils import get_moving_links, get_self_link_pairs, pairwise_collision, pairwise_link_collision
from itertools import product
from full_demo import create_env, INITIAL_POSITION, get_ik_sol, set_ja_goal


import pybullet as p
import time

In [ ]:
# Create a real environment.
physicsClient, robotId, ll, ul, jr, bounds, tableheight = create_env(with_block=False)

In [ ]:
# Let's create a desired set of goals.
current_ja = INITIAL_POSITION
down = [1, 0, 0, 0]
goal_ja = get_ik_sol([0, 0, tableheight+0.1], down, ll, ul, jr, robotId)

In [ ]:
from ompl import base as ob
from ompl import geometric as og

## Create the state space

In [ ]:
n_joints = len(current_ja)

# The state space we're trying to plan in is JOINT SPACE.
# We could probably also perform things in SE(3), but maybe not yet.
state_space = ob.RealVectorStateSpace(n_joints)

# Describe the general bounds of the optimization problem.
obbounds = ob.RealVectorBounds(n_joints)
for i, (low, hi) in enumerate(bounds):
    obbounds.setHigh(i, hi)
    obbounds.setLow(i, low)
state_space.setBounds(obbounds)

## Create the state validity checker

In [ ]:
class StateChecker:
    def __init__(self):
        self.simID, self.robotId, tableId = create_state_checker_env()
        obstacles = [tableId]

        # DON'T WORRY ABOUT THIS SECTION; THIS IS JUST BOILERPLATE FOR GETTING WHICH PAIRS
        # OF LINKS TO CHECK.
        joint_idx = list(range(7))
        self.check_link_pairs = get_self_link_pairs(self.robotId, joint_idx, simID=self.simID)
        moving_links = frozenset(get_moving_links(self.robotId, joint_idx, simID=self.simID))
        moving_bodies = [(self.robotId, moving_links)]
        self.check_body_pairs = list(product(moving_bodies, obstacles))

    def is_state_valid(self, state):
        state_list = [state[i] for i in range(7)]

        # Put our simulation into the specified state.
        for joint, value in zip(range(7), state_list):
            p.resetJointState(self.robotId, joint, value, targetVelocity=0, physicsClientId=self.simID)

        # Check for collisions with the robot links.
        for link1, link2 in self.check_link_pairs:
            if pairwise_link_collision(self.robotId, link1, self.robotId, link2, simID=self.simID):
                return False

        # Check for collisions with the environment.
        for body1, body2 in self.check_body_pairs:
            if pairwise_collision(body1, body2, self.simID):
                return False
        return True
checker = StateChecker()

## Create the OMPL Solver

In [ ]:
# Create the solver. The default planner is RRT, but we can set it to any
# planning algorithm (or create our own!) via `solver.setPlanner`
solver = og.SimpleSetup(state_space)
solver.setStateValidityChecker(ob.StateValidityCheckerFn(checker.is_state_valid))

# Initialize the start and the goal.
start, goal = ob.State(state_space), ob.State(state_space)
for i, (s_i, e_i) in enumerate(zip(current_ja[:7], goal_ja[:7])):
    start[i] = s_i
    goal[i] = e_i
solver.setStartAndGoalStates(start, goal)

In [ ]:
# Solve the task with some time constraints (not sure exactly what this means...)
solved = solver.solve(DEFAULT_PLANNING_TIME)
if not solved:
    raise ValueError("unable to solve the problem")

## Postprocessing

In [ ]:
# Simplify, essentially tightening the solution.
solver.simplifySolution()

# Get the solution, and perform interpolation.
path = solver.getSolutionPath()
path.interpolate(INTERPOLATE_NUM)
path_states = path.getStates()
path_list = [[state[i] for i in range(n_joints)] for state in path_states]

## Let's try and execute the path!

In [ ]:
for i in range (len(path_list)):
    set_ja_goal(path_list[i][:7], robotId)
    p.stepSimulation()
    time.sleep(1./240.)

for i in range(10000):
    p.stepSimulation()
    time.sleep(1./240.)